# Intro

This notebook summarizes the experimental results obtained using Kedro pipeline and presents static figures as well as model scores.

It contains the following sections:
1. Data loading - loading the data from the Kedro catalog: model metrics from cross-validation runs.
2. Data inspection - visualizing the data using boxplots.
3. Statistical analysis - performing statistical analysis of the data using Kruskal-Wallis test and post-hoc pairwise tests.

# Lib imports

In [2]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [3]:
import pandas as pd
import pingouin as pg
import plotly as py
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns

import IPython.display as dd

from deep_hybrid_recommender.pipelines.experiment.nodes import perform_statistical_comparison

In [4]:
%reload_kedro

[05/29/23 20:59:59] INFO     Resolved project path as:                                              ]8;id=734374;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=591546;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             /home/fwojcik/projects/deep-hybrid-recommenders.                                      
                             To set a different path, run '%reload_kedro <project_root>'                           

                    WARNING  /home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/ked ]8;id=523149;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/warnings.py\warnings.py]8;;\:]8;id=610499;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/warnings.py#109\109]8;;\
                             ro/framework/startup.py:98: DeprecationWarning: project_version in                    
                             pyproject.toml is deprecated, use kedro_init_version instead                          
                               warnings.warn(                                                                      
                                                                                                                   

[05/29/23 20:59:59] INFO     Kedro project Deep Hybrid Recommender                                  ]8;id=232503;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=400990;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=134354;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=79477;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=967832;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=870558;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/ipython/__init__.py#110\110]8;;\

# Load data

In [5]:
metrics_to_show = ['val_MAPE', 'val_MAE', 'val_MSE']

In [15]:
colab_filt_val_metrics = catalog.load("experiment.colab_filtering_crossval_val_metrics")
deep_colab_filt_val_metrics = catalog.load("experiment.deep_colab_filtering_crossval_val_metrics")
hybrid_rec_val_metrics = catalog.load("experiment.deep_hybrid_rec_crossval_val_metrics")
gnn_rec_val_metrics = catalog.load("experiment.gnn_rec_crossval_val_metrics")

val_metrics = pd.concat([colab_filt_val_metrics, deep_colab_filt_val_metrics, hybrid_rec_val_metrics, gnn_rec_val_metrics], axis=0, ignore_index=True)
val_metrics = pd.melt(val_metrics, id_vars='model_name', var_name='metric')
val_metrics = val_metrics.loc[val_metrics.metric.str.contains("val_")]

[05/29/23 21:07:16] INFO     Loading data from                                                  ]8;id=569783;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=590379;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             'experiment.colab_filtering_crossval_val_metrics' (CSVDataSet)...                     

                    INFO     Loading data from                                                  ]8;id=890709;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=916762;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             'experiment.deep_colab_filtering_crossval_val_metrics'                                
                             (CSVDataSet)...                                                                       

                    INFO     Loading data from                                                  ]8;id=661573;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=235803;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             'experiment.deep_hybrid_rec_crossval_val_metrics' (CSVDataSet)...                     

                    INFO     Loading data from 'experiment.gnn_rec_crossval_val_metrics'        ]8;id=684383;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=131016;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (CSVDataSet)...                                                                       

# Validation metrics inspection

In [18]:
val_metrics.pivot_table(index='model_name', columns='metric', aggfunc='mean')

value                     
metric                 val_MAE  val_MAPE    val_MSE
model_name                                         
colab filtering       3.752275  0.813148  17.231289
deep colab filtering  0.476319  0.157380   0.512165
gnn recommender       0.157461  0.044626   0.129898
hybrid recommender    0.166640  0.062050   0.206777

## Visualization of validation metrics

In [19]:
fig = px.box(
    val_metrics.loc[val_metrics.metric.isin(metrics_to_show)],
    color='model_name',
    y='value',
    facet_col='metric',
    title='Validation metrics comparison')
fig.write_html("val_metrics.html")
fig.write_image("val_metrics.png")
fig.show()

<img src="val_metrics.png">

Visual inspection of the plots indicate three metric values are the smallest for the GNN recommender model. Of three approaches considered in the study, the classic collaborative filtering model scored the worst results every time.

## Statistical comparisons of metrics

Considering the fact that for repeated k-fold cross-validation samples are not independent (instead: selected on purpose to be in training or test subset without duplication) and measures are repeated k-times for multiple classifiers, Kruskal test was selected as a non-parametric rank-based alternative to classic ANOVA.

For each metric, the following two tests are performed:

1. The test for **overall** differences:
    1. **H0** - mean ranks of the groups are the same.
    2. **HA** - mean ranks of the groups are not the same
2. Post-hoc test for pairwise differences.

In [20]:
comparison_metrics = {metric: perform_statistical_comparison(val_metrics, metric) for metric in metrics_to_show}

In [21]:
for metric, (overall_result, pairwise_result) in comparison_metrics.items():
    dd.display(dd.Markdown(f"### {metric} analysis"))
    dd.display(dd.Markdown(f"#### Overall test"))
    dd.display(overall_result)
    dd.display(dd.Markdown(f"#### Pairwise tests"))
    display(pairwise_result)

### val_MAPE analysis

#### Overall test

,Source,ddof1,H,p-unc
Kruskal,model_name,3,33.993659,1.987304e-07


#### Pairwise tests

,Contrast,A,B,Paired,Parametric,U-val,alternative,p-unc,p-corr,p-adjust,hedges
0,model_name,colab filtering,deep colab filtering,False,False,100.0,two-sided,0.000183,0.001096,bonferroni,25.091008
1,model_name,colab filtering,gnn recommender,False,False,100.0,two-sided,0.000183,0.001096,bonferroni,34.241320
2,model_name,colab filtering,hybrid recommender,False,False,100.0,two-sided,0.000183,0.001096,bonferroni,29.253058
3,model_name,deep colab filtering,gnn recommender,False,False,100.0,two-sided,0.000183,0.001096,bonferroni,5.285611
4,model_name,deep colab filtering,hybrid recommender,False,False,100.0,two-sided,0.000183,0.001096,bonferroni,3.858011
5,model_name,gnn recommender,hybrid recommender,False,False,23.0,two-sided,0.045155,0.270927,bonferroni,-0.839019


...

### val_MSE analysis

#### Overall test

,Source,ddof1,H,p-unc
Kruskal,model_name,3,34.254146,1.750928e-07


#### Pairwise tests

,Contrast,A,B,Paired,Parametric,U-val,alternative,p-unc,p-corr,p-adjust,hedges
0,model_name,colab filtering,deep colab filtering,False,False,100.0,two-sided,0.000183,0.001096,bonferroni,51.812591
1,model_name,colab filtering,gnn recommender,False,False,100.0,two-sided,0.000183,0.001096,bonferroni,53.825255
2,model_name,colab filtering,hybrid recommender,False,False,100.0,two-sided,0.000183,0.001096,bonferroni,52.878386
3,model_name,deep colab filtering,gnn recommender,False,False,100.0,two-sided,0.000183,0.001096,bonferroni,5.031492
4,model_name,deep colab filtering,hybrid recommender,False,False,99.0,two-sided,0.000246,0.001477,bonferroni,3.316019
5,model_name,gnn recommender,hybrid recommender,False,False,18.0,two-sided,0.017257,0.103545,bonferroni,-1.055765


Analysis of the presented results indicates the following:
1. There is a significant difference in the performance of more advanced models (hybrid recommender and GNN recommender), compared to collaborative filtering and deep collaborative filtering.
2. There is no significant difference between the deep hybrid model and the GNN model.

# Test metrics analysis

In [22]:
test_met_dicts = []
for catalog_name in [met for met in catalog.list() if 'experiment.' in met and 'test' in met]:
    _, metric = catalog_name.split(".")
    model = metric.replace("_test_metrics", "")
    met_dict = catalog.load(catalog_name)
    met_dict['model'] = model
    
    test_met_dicts.append(met_dict)

test_mets_df = pd.DataFrame.from_records(test_met_dicts, index=list(range(len(test_met_dicts))))
test_mets_df

[05/29/23 21:08:12] INFO     Loading data from                                                  ]8;id=95666;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=803851;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             'experiment.collaborative_filtering_test_metrics'                                     
                             (MetricsDataSet)...                                                                   

                    INFO     Loading data from                                                  ]8;id=543535;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=526170;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             'experiment.deep_collaborative_filtering_test_metrics'                                
                             (MetricsDataSet)...                                                                   

                    INFO     Loading data from 'experiment.deep_hybrid_rec_test_metrics'        ]8;id=808315;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=959712;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (MetricsDataSet)...                                                                   

                    INFO     Loading data from 'experiment.gnn_rec_test_metrics'                ]8;id=873796;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=154650;file:///home/fwojcik/anaconda3/envs/py39clean/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (MetricsDataSet)...                                                                   

,test_MSE,test_MAPE,test_MAE,model
0,17.617960,0.814397,3.798708,collaborative_filtering
1,0.527138,0.156110,0.463374,deep_collaborative_filtering
2,0.192739,0.052388,0.146859,deep_hybrid_rec
3,0.153372,0.059262,0.166875,gnn_rec


# Conclusions

Analysis of all overall comparisons for all metrics allows to reject the null hypothesis. 
**This means that the mean ranks of the groups are not the same.**

**The post-hoc test results show that the Hybrid Recommender and GNN recommender model are significantly better than the other two models in all cases.**

Best test result were obtained by the **Hybrid recommender model** (MAPE - 0.7 perc.point better compared to GNN, MAE - 2 perc. points better compared to GNN) but in terms of MSE metric, **GNN recommender** scored best, improving by 4 percentage points over Hybrid Recommender.